# Using CLDK to generate JUnit tests

In this tutorial, we will use [CLDK](https://github.com/IBM/codellm-devkit/tree/main) to implement a simple unit test generator for Java. You'll explore some of the benefits of using CLDK to perform quick and easy program analysis and build an LLM-based test generator. By the end of this tutorial, you will have implemented such a tool and generated a JUnit test case for a Java application.

Specifically, you will learn how to perform the following tasks on the application under test to create LLM prompts for test generation:

1. Create a new instance of the CLDK class.
2. Create an analysis object for the Java application under test.
3. Iterate over all files in the application.
4. Iterate over all classes in a file.
5. Iterate over all methods in a class.
6. Get the code body of a method.
7. Get the constructors of a class.
<!-- 7. Initialize treesitter utils for the class file content.
8. Sanitize the class for analysis. -->

We will write a couple of helper methods to (1) format the LLM instruction for generating test cases for a given focal method (i.e., method under test) and (2) prompt the LLM via Ollama. We will then use CLDK to go through an application and generate unit test cases for the target method.

## Prequisites

See the [Code Sumarization](./code_summarization.ipynb) recipe, which describes the prerequisites and set up that are required for this notebook, as well. These prerequisites include Python 3.11 or later, Java 11 or later, Maven 3.9 or later, [Ollama 0.3.4](https://ollama.com/) or later, [Granite code models](https://ollama.com/library/granite-code), a sample Java application (the [Apache Commons CLI](https://github.com/apache/commons-cli), and the CKDK tools.

## Build a JUnit test generator using CLDK and Granite Code Model

Let's build a JUnit test generator using CLDK and the Granite Code Instruct Model.

Generating unit tests for code is an important task and developers often have to put in significant effort in writing good test cases. There are various tools available for automated test generation, such as EvoSuite, which uses evolutionary algorithms to generate unit test cases for Java. However, the generated test cases are not natural and often developers do not prefer to add them to their test suites. LLMs, having been trained with developer-written code, have a better affinity towards generating more natural code, code that is more readable, comprehensible, and maintainable. In this excercise, we will show how we can leverage LLMs to generate test cases with the help of CLDK. 

For simplicity, we will cover certain aspects of test generation and provide some context information to the LLM to help it create usable test cases. In this exercise, we will generate a unit test for a non-private method from a Java class and provide the focal method body and the signature of all the constructors of the class so that LLM can understand how to create an object of the focal class during the setup phase of the tests.
<!-- Also, we will ask LLMs to generate ```N``` number of test cases, where ```N``` is the cyclomatic complexity of the focal method. The intuition is that one test may not be sufficient for covering fairly complex methods, and a cyclomatic complexity score can provide some guidance towards that.  -->

Step 1: Import the required modules.

In [ ]:
import ollama
from cldk import CLDK
from cldk.analysis import AnalysisLevel

Step 2: Define a function for creating the LLM prompt, which instructs the LLM to generate unit tests cases and includes signatures of relevant constructors and the body of the focal method.

In [ ]:
def format_inst(focal_method_body, focal_method, focal_class, constructor_signatures, language):
    """
    Format the LLM instruction for the given focal method and class.
    """
    inst = f"Question: Can you generate junit tests with @Test annotation for the method `{focal_method}` in the class `{focal_class}` below. Only generate the test and no description.\n"
    inst += 'Use the constructor signatures to form the object if the method is not static. Generate the code under ``` code block.'
    inst += "\n"
    inst += f"```{language}\n"
    inst += f"public class {focal_class} " + "{\n"
    inst += f"{constructor_signatures}\n"
    inst += f"{focal_method_body} \n" 
    inst += "}"
    inst += "```\n"
    inst += "Answer:\n"
    return inst

Step 3: Define a function to call the LLM (in this case, Granite code 8b-instruct) using Ollama.

In [ ]:
def prompt_ollama(message: str, model_id: str = "granite-code:8b-instruct") -> str:
    """Prompt local model on Ollama"""
    response_object = ollama.generate(model=model_id, prompt=message, options={"temperature":0.2})
    return response_object["response"]

Step 4: Collect the relevant information for the focal method and prompt the LLM. To do this, we go through all the classes in the application, and for each class, we collect the signatures of its constructors. If a class has no constructors, we add the signature of the default constructor. Then, we go through each non-private method of the class and formulate the prompt using the constructor and the method information. Finally, we use the prompt to call LLM to generate test cases and get the LLM response. If analysis has been run already, CLDK will use the existing analysis.

In [ ]:
# Create an instance of CLDK for Java analysis
cldk = CLDK(language="java")

# Create an analysis object for the Java application. Provide the application path.
analysis = cldk.analysis(project_path="temp/commons-cli-rel-commons-cli-1.7.0", analysis_level=AnalysisLevel.symbol_table, analysis_json_path='analysis')

# For simplicity, we run the test generation on a single focal class and method (this filter can be removed to run this code over the entire application)
focal_class = "org.apache.commons.cli.GnuParser"
focal_method = "flatten(Options, String[], boolean)"

# Go through all the classes in the application
for class_name in analysis.get_classes():

    if class_name == focal_class:
        print(f"Class: {class_name}")
        class_details  = analysis.get_class(qualified_class_name=class_name)
        focal_class_name = class_name.split(".")[-1]

        # Generate test cases for non-interface and non-abstract classes
        if not class_details.is_interface and "abstract" not in class_details.modifiers:
            
            # Get all constructor signatures
            constructor_signatures = ""
            
            for method in analysis.get_methods_in_class(qualified_class_name=class_name):
                method_details = analysis.get_method(qualified_class_name=class_name, qualified_method_name=method)
                
                if method_details.is_constructor:
                    constructor_signatures += method_details.signature + '\n'
            
            # If no constructor present, then add the signature of the default constructor
            if constructor_signatures == "":
                constructor_signatures = f"public {focal_class_name}() " + "{}"
            
            # Go through all the methods in the class
            for method in analysis.get_methods_in_class(qualified_class_name=class_name):               
                if method == focal_method:
                    # Get the method details
                    method_details = analysis.get_method(qualified_class_name=class_name, qualified_method_name=method)
                    
                    # Generate test cases for non-private methods
                    if "private" not in method_details.modifiers and not method_details.is_constructor:
                        
                        # Gather all the information needed for the prompt, which are focal method body, focal method name, focal class name, and constructor signature
                        prompt = format_inst(
                            focal_method_body=method_details.declaration+method_details.code,
                            focal_method=method.split("(")[0],
                            focal_class=focal_class_name,
                            constructor_signatures=constructor_signatures,
                            language="java"
                        )
                        
                        # Print the instruction
                        print(f"Instruction:\n{prompt}\n")
                        print(f"Generating test case and it will take few minutes (or even seconds) based on where the model has been hosted...\n")
                        
                        # Prompt the local model on Ollama
                        llm_output = prompt_ollama(message=prompt)
                        
                        # Print the LLM output
                        print(f"LLM Output:\n{llm_output}")

Note that a file [analysis.json](./analysis/analysis.json) was used from existing ```analysis``` folder.

After the LLM's response is received, you should see the generated test case for the `flatten` method printed out (similar to the following).

**LLM Output:**
```java
import static org.junit.jupiter.api.Assertions.assertEquals;

import java.util.ArrayList;
import java.util.List;

import org.apache.commons.cli.Options;
import org.junit.jupiter.api.Test;

class GnuParserTest {

    @Test
    void testFlatten() {
        final Options options = new Options();
        final String[] arguments = {};
        final boolean stopAtNonOption = false;

        final String[] expected = {};
        final String[] actual = GnuParser.flatten(options, arguments, stopAtNonOption);

        assertEquals(expected, actual);
    }
}
```